In [3]:
source = "GCD.txt"
instructions = "GCD_Instructions.coe"
data = "GCD_Data.coe"

In [4]:
import re

In [5]:
registers = {
    "$zero":"00000",
    "$at" : "00001",
    "$v0" : "00010",
    "$v1" : "00011",
    "$a0" : "00100",
    "$a1" : "00101",
    "$a2" : "00110",
    "$a3" : "00111",
    "$t0" : "01000",
    "$t1" : "01001",
    "$t2" : "01010",
    "$t3" : "01011",
    "$t4" : "01100",
    "$t5" : "01101",
    "$t6" : "01110",
    "$t7" : "01111",
    "$s0" : "10000", 
    "$s1" : "10001",
    "$s2" : "10010",
    "$s3" : "10011",
    "$s4" : "10100",
    "$s5" : "10101",
    "$s6" : "10110",
    "$s7" : "10111",
    "$t8" : "11000",
    "$t9" : "11001",
    "$k0" : "11010",
    "$k1" : "11011",
    "$gp" : "11100",
    "$fp" : "11101",
    "$sp" : "11110",
    "$ra"  :"11111"       
    
}

In [6]:
opCode = {
    "lw"   : "000001",
    "sw"   : "000010",
    "addi" : "000011",
    "compi": "000100",
    "b"    : "000101",
    "bl"   : "000110",
    "bcy"  : "000111",
    "bncy" : "001000",
    "br"   : "001001",
    "bltz" : "001010",
    "bz"   : "001011",
    "bnz"  : "001100",
    "add"  : "000000",
    "comp" : "000000",
    "and"  : "000000",
    "xor"  : "000000",
    "shll" : "000000",
    "shrl" : "000000",
    "shllv": "000000",
    "shrlv": "000000",
    "shra" : "000000",
    "shlav": "000000",
}

functionCode = {
    "add"  : "0000000000",
    "comp" : "0000000001",
    "and"  : "0000000010",
    "xor"  : "0000000011",
    "shll" : "0000000100",
    "shrl" : "0000000101",
    "shllv": "0000000110",
    "shrlv": "0000000111",
    "shra" : "0000001000",
    "shlav": "0000001001",
}

shamt =  [
    "0000000100", "0000000101", "0000001000",    
]

In [7]:
def OPcode_Functioncode(operation):    
    opcode = None
    functioncode = None
    if operation in opCode:
        opcode = opCode[operation]
    if opcode == "000000":
        functioncode = functionCode[operation]
    return opcode, functioncode



In [14]:
def Encoder(res):
    operation = res[0]
    binaryOutput = ""
    op,func = OPcode_Functioncode(operation)

    try:
        if op == "000000":
            reg1 = res[1]
            sh = res[2]
            if func in shamt:            
                binaryOutput = op + registers['$' + reg1] + "00000" + '{0:06b}'.format(int(sh)) + func
            else:
                binaryOutput = op + registers['$' + reg1] + registers['$' + sh] + "000000" + func

        case = int(op, 2)
        #print(case)
        if case == 1:
            binaryOutput = op + registers['$' + res[3] ] + registers['$' + res[1]] + '{0:016b}'.format(int(res[2]) & 65535)

        elif case == 2:
            binaryOutput = op + registers['$' + res[3] ] + registers['$' + res[1]] + '{0:016b}'.format(int(res[2]) & 65535)

        elif case == 3:
            binaryOutput = op + registers['$' + res[1] ] + "00000" + '{:016b}'.format(int(res[2])& 65535)

        elif case == 4:
            binaryOutput = op + registers['$' + res[1] ] + "00000" + '{:016b}'.format(int(res[2])& 65535)

        elif case == 5:
            binaryOutput = op + '{0:026b}'.format(int(res[1]) & 67108863)

        elif case == 6:
            binaryOutput = op + '{0:026b}'.format(int(res[1]) & 67108863 ) 

        elif case == 7:
            binaryOutput = op + '{0:026b}'.format(int(res[1]) & 67108863)

        elif case == 8:
            binaryOutput = op + '{0:026b}'.format(int(res[1]) & 67108863)
            
        elif case == 9:
            binaryOutput = op + registers['$' + res[1] ] + "000000000000000000000"

        elif case == 10:
            binaryOutput = op + registers['$' + res[1] ] + '{:021b}'.format(int(res[2]) & 2097151)

        elif case == 11:
            binaryOutput = op + registers['$' + res[1] ] + '{:021b}'.format(int(res[2]) & 2097151)

        elif case == 12:
            binaryOutput = op + registers['$' + res[1] ] + '{:021b}'.format(int(res[2]) & 2097151)
            
            
    except:
        binaryOutput= "error"
    
    return binaryOutput
    

def get_labels(source):
    labels = {}
    file2 = open('target.txt', 'w')

    file1 = open(source, 'r')
    count = 0

    while True:
        line = file1.readline()
        if not line:
            break
        if not line.strip():
            continue

        res = re.findall(r'\w+', line)
        op,func = OPcode_Functioncode(res[0])
        
        if op == None:
            if ':' in line:
                res = re.findall(r'\w+', line)
                label = res[0]
                labels[label] = count
#                 if len(res) > 1:
#                     op,func = OPcode_Functioncode(res[1])
#                     res =res[1:]
#                 print(res)
#                 if op != None:
#                     for i in res:
#                         file2.writelines(i + " ")
#                     file2.writelines("\n")
#                 else:
#                     print("Syntax error")
                
            else:
                print("Syntax error")
        else:
            file2.writelines(str(count)+ " " + line.strip() + "\n")
            count += 1

    file1.close()
    file2.close()
    return labels, count    
            
        

In [15]:
labels, count = get_labels(source)
file1 = open('target.txt', 'r')
dest = open(instructions, 'w')
dest.writelines("memory_initialization_radix = 2;" + 
               "\n" + "memory_initialization_vector= " + "\n" + 
               "00000000000000000000000000000000," + "\n")
while True:
    line = file1.readline()
    if not line:
        break
    res = re.findall(r'\w+', line)
    no = int(res[0])
    res = res[1:]
    c = int(opCode[res[0]], 2)
    if c == 0 or (c <=12 and c>=9) or c == 3 or c == 4:
        res = res[:3]
    elif c>=5 and c<=8:
        res = res[:2]
    else:
        res = res[:4]
    if c== 3 or c==4:
        if '-' in line:
            res[2] = -int(res[2])
    if c <= 8 and c >= 5:
        res[1]=labels[res[1]]-no -1;
    elif c >= 10 and c <= 12:
        res[2]=labels[res[2]]-no -1;
    print(res)
    if count == 1 :
        dest.writelines(Encoder(res) + ";")
        break
    
    dest.writelines(Encoder(res)+","+"\n")
    count = count-1
   

file1.close()
dest.close()

['lw', 't0', '0', 'zero']
['lw', 't1', '4', 'zero']
['bz', 't0', 15]
['bz', 't1', 11]
['and', 't2', 'zero']
['add', 't2', 't1']
['comp', 't3', 't0']
['add', 't2', 't3']
['bltz', 't2', 3]
['comp', 't3', 't0']
['add', 't1', 't3']
['b', -9]
['comp', 't4', 't1']
['add', 't0', 't4']
['b', -12]
['and', 'v0', 'zero']
['add', 'v0', 't0']
['b', 3]
['and', 'v0', 'zero']
['add', 'v0', 't1']
['b', 0]
['b', -1]


In [10]:
num = [ 36, 156  ]

dest = open(data, 'w')
dest.writelines("memory_initialization_radix = 2;" + 
               "\n" + "memory_initialization_vector= " + "\n")

for i in num:
    p =  '{:032b}'.format(i & 4294967295 )
    dest.writelines(p +"," + "\n")
dest.writelines(";")
dest.close()

In [19]:

file2 = open('stage3.txt', 'w')

# Using readline()
file1 = open('stage2.txt', 'r')

while True:
    line = file1.readline()
    if not line:
        break
    if not line.strip():
        continue

    res = re.findall(r'\w+', line)
    c = int(opCode[res[1]], 2)
    
    if c == 0 or (c <=12 and c>=9) or c == 3 or c == 4:
        res = res[:4]
    elif c>=5 and c<=8:
        res = res[:3]
    else:
        res = res[:5]
    if c== 3 or c==4:
        if '-' in line:
            res[3] = -int(res[3])
    if c <= 8 and c >= 5:
        res[2]=labels[res[2]];
    elif c >= 10 and c <= 12:
        res[3]=labels[res[3]];
    print(res)
    for r in res:
        file2.writelines(str(r) + " ")
    file2.writelines("\n")
    

file1.close()
file2.close()

['0', 'lw', 't0', '0', 'zero']
['1', 'lw', 't1', '4', 'zero']
['2', 'bz', 't0', 18]
['3', 'bz', 't1', 15]
['4', 'and', 't2', 'zero']
['5', 'add', 't2', 't1']
['6', 'comp', 't3', 't0']
['7', 'add', 't2', 't3']
['8', 'bltz', 't2', 12]
['9', 'comp', 't3', 't0']
['10', 'add', 't1', 't3']
['11', 'b', 3]
['12', 'comp', 't4', 't1']
['13', 'add', 't0', 't4']
['14', 'b', 3]
['15', 'and', 'v0', 'zero']
['16', 'add', 'v0', 't0']
['17', 'b', 21]
['18', 'and', 'v0', 'zero']
['19', 'add', 'v0', 't1']
['20', 'b', 21]
['21', 'b', 21]
